In [5]:
# You still need (if RAG) a vector database. Using this code as a reminder. Jump to block 4 below unless you need to reinstall the database.
import os
import warnings
import logging
import time
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Optional: Disabling warnings
warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO)

# Configuration
FILE_PATH = "docs/War-of-the-Worlds.pdf"
CHROMA_PERSIST_DIR = "chroma_store_chatbot"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

# Function to load and split the PDF document
def load_and_split_document(filepath):
    """Load the PDF file, split it into chunks, and return chunks with metadata."""
    start_time = time.time()
    
    loader = PyPDFLoader(filepath)
    documents = loader.load()  # This returns a list of Document objects
    logging.info(f"Document loaded in {time.time() - start_time:.2f} seconds.")
    
    # Split document into chunks of 1000 characters with a 100-character overlap
    start_time = time.time()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    split_documents = text_splitter.split_documents(documents)  # Pass the list directly
    logging.info(f"Document split into chunks in {time.time() - start_time:.2f} seconds.")

    # Add metadata to each chunk
    for doc in split_documents:
        doc.metadata["source"] = filepath
    
    return split_documents

# Embed and store the document if not already processed
def embed_and_store_document():
    """Load, embed, and store the document in Chroma."""
    logging.info(f"Processing document: {FILE_PATH}")
    
    # Initialize the Ollama embedding function
    embeddings = OllamaEmbeddings(model="llama3")
    
    # Initialize or load the existing Chroma vectorstore with the embedding function
    start_time = time.time()
    vectorstore = Chroma(persist_directory=CHROMA_PERSIST_DIR, embedding_function=embeddings)
    logging.info(f"Chroma vectorstore initialized in {time.time() - start_time:.2f} seconds.")
    
    # Check if the document has already been processed
    existing_files = set(metadata.get("source") for metadata in vectorstore.get()["metadatas"])
    
    if FILE_PATH in existing_files:
        logging.info(f"Document already processed: {FILE_PATH}")
        return
    
    # Load and split the document into chunks
    document_chunks = load_and_split_document(FILE_PATH)
    
    # Embed and add the new document's chunks to the vectorstore
    start_time = time.time()
    vectorstore.add_documents(document_chunks)
    logging.info(f"Document embedded and added to vectorstore in {time.time() - start_time:.2f} seconds.")
    
    # Persist the updated vector store
    start_time = time.time()
    vectorstore.persist()
    logging.info(f"Chroma vectorstore persisted in {time.time() - start_time:.2f} seconds.")
    
    logging.info("Processing complete.")

# Run the embedding and storing process for a single document
embed_and_store_document()


/Users/jerhenry/Documents/Workdoc/f-Virtual_Machines/Environments/Pytorch/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
INFO:root:Processing document: docs/War-of-the-Worlds.pdf
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:root:Chroma vectorstore initialized in 0.26 seconds.
INFO:root:Document loaded in 0.48 seconds.
INFO:root:Document split into chunks in 0.00 seconds.
INFO:root:Document embedded and added to vectorstore in 762.85 seconds.
INFO:root:Chroma vectorstore persisted in 0.02 seconds.
INFO:root:Processing complete.


In [3]:
# In case of need, cleaning and resetting the chromadb so you can do this exercize multiple times
import shutil
import os

# Configuration
CHROMA_PERSIST_DIR = "chroma_store_chatbot"

def clear_chroma_database():
    """Delete the Chroma database directory to start from scratch."""
    if os.path.exists(CHROMA_PERSIST_DIR):
        # Remove the entire directory and its contents
        shutil.rmtree(CHROMA_PERSIST_DIR)
        print(f"Chroma database at '{CHROMA_PERSIST_DIR}' has been cleared.")
    else:
        print(f"No Chroma database found at '{CHROMA_PERSIST_DIR}' to clear.")

# Run the function to clear the Chroma database
clear_chroma_database()


Chroma database at 'chroma_store_chatbot' has been cleared.


In [6]:
#We still need these bricks, so do not run this part of the notebook in isolation
persist_directory = "chroma_store_chatbot"
embedding = OllamaEmbeddings(model="llama3")
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [7]:
# Let's start here, by checking that the vectordb still has the 443 chunks of the pdf document.
print(vectordb._collection.count())

443


In [8]:
# And le'ts bring back the context from the previous notebook.
question = "Did the spaceship come from the planet Mars?"
docs = vectordb.max_marginal_relevance_search(question,k=2, fetch_k=3)
#Using Llama3 as the LLM, and Ollama as the wrapper to interact with Llama3
from langchain_community.llms import Ollama
llm = Ollama(model = "llama3")

In [9]:
#!pip install ollama
#!ollama serve & ollama pull llama3 & ollama pull nomic-embed-text
#!pip install ollama langchain beautifulsoup4 chromadb gradio -q

In [10]:
# Let's wrap the RAG into a simple UI. 
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain_community.embeddings import OllamaEmbeddings

# Create Ollama embeddings and vector store
#embeddings = OllamaEmbeddings(model="nomic-embed-text")
#vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    # Explicitly create a new conversation with only the current prompt
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='llama3', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


# Define the RAG setup
retriever = vectordb.as_retriever()

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)

# Define the Gradio interface
def get_important_facts(question):
    return rag_chain(question)

# Create a Gradio app interface
iface = gr.Interface(
  fn=get_important_facts,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3",
  description="Ask questions",
)

# Launch the Gradio app
iface.launch()
# example q: did the aliens eventually go on to land on Venus?

INFO:httpx:HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:7867/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7867/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


In [11]:
# Now, Ollama re-injects the previous question and asnwer into the model with the next question. But other LLMs would forget the previous question (remember Dolly?) You can add memory with a memory module.
# Let's also add a debug function to show what was passed to the LLM and the retriever.
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.runnables import Runnable

# Define the prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 

{context}

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Define memory to store the previous exchanges
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Custom Runnable LLM class with debugging
class OllamaLLM(Runnable):
    def __init__(self, llm_fn):
        self.llm_fn = llm_fn

    def invoke(self, input, config=None, **kwargs):
        # If the input is a StringPromptValue or similar object, treat it as a string
        question = str(input)  # Convert the input to a string if necessary
        context = kwargs.get("context", "")  # Retrieve context from kwargs if available

        # Print what was passed to the LLM
        print(f"Question passed to LLM: {question}")
        print(f"Context passed to LLM: {context}")

        # Handle additional kwargs such as stop, if needed
        stop = kwargs.get("stop", None)

        # If 'stop' or other arguments need to be passed to the LLM function, handle them here
        response = self.llm_fn(question, context)
        
        # Print the response from the LLM
        print(f"Response from LLM: {response}")

        return response

    def predict(self, input, **kwargs):
        return self.invoke(input, **kwargs)

    def __call__(self, *args, **kwargs):
        return self.invoke(*args, **kwargs)

# Instantiate the custom LLM class
ollama_llm_instance = OllamaLLM(ollama_llm)

# Define the conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ollama_llm_instance,
    retriever=retriever,
    memory=memory # add the memory module, to pass the previous exchange to the LLM as well
)

# Define the function to get important facts with debugging
def get_important_facts(question):
    # Print what is passed to the retriever
    print(f"Question passed to retriever: {question}")
    
    # Run the chain and capture the memory state
    response = qa_chain.run({"question": question})
    
    # Print what is in memory after the retrieval
    print(f"Memory state: {memory.buffer}")

    return response

# Create a Gradio app interface
iface = gr.Interface(
    fn=get_important_facts,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="RAG with Llama3",
    description="Ask questions",
)

# Launch the Gradio app
iface.launch()



INFO:httpx:HTTP Request: GET http://127.0.0.1:7868/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7868/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## Chains and Tools

You may also need your LLM to retrieve information from elsewhere. This is where LangChain tools and chains become useful.

In [26]:
# Let's define a simple function that returns the weather for a location
import sys
import os
import openai
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenWeatherMap API key from the environment variables
OPENWEATHER_API_KEY = os.environ.get('OPENWEATHER_API_KEY') # set a keys.env file in the parent directory, and set there your openweather API key

from langchain.agents import tool
import requests
from pydantic import BaseModel, Field

# Define the input schema
class CityInput(BaseModel):
    city: str = Field(..., description="City name to fetch weather data for")

# Tool to get the current weather
@tool(args_schema=CityInput)
def get_current_weather(city: str) -> dict:
    """Fetch current weather for a given city."""

    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
    
    # Parameters for the weather request
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric',  # To get the temperature in Celsius
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        weather_data = {
            "City": city,
            "Temperature": f"{results['main']['temp']}°C",
            "Weather Description": results['weather'][0]['description'],
            "Humidity": f"{results['main']['humidity']}%",
            "Wind Speed": f"{results['wind']['speed']} m/s",
            "Pressure": f"{results['main']['pressure']} hPa"
        }
    else:
        raise Exception(f"Weather API Request failed with status code: {response.status_code}")
    
    return weather_data



In [27]:
get_current_weather("Richmond")

{'City': 'Richmond',
 'Temperature': '23.22°C',
 'Weather Description': 'broken clouds',
 'Humidity': '73%',
 'Wind Speed': '4.12 m/s',
 'Pressure': '1015 hPa'}

In [28]:
# Even without an LLM, we can of course build a nice sentence for the response
from langchain.agents import tool
import requests
from pydantic import BaseModel, Field

# Define the input schema
class CityInput(BaseModel):
    city: str = Field(..., description="City name to fetch weather data for")

# Tool to get the current weather
@tool(args_schema=CityInput)
def get_current_weather(city: str) -> dict:
    """Fetch current weather for a given city."""
    
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
    
    # Parameters for the weather request
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric',  # To get the temperature in Celsius
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        temperature = results['main']['temp']
        weather_description = results['weather'][0]['description']
        humidity = results['main']['humidity']
        wind_speed = results['wind']['speed']
        pressure = results['main']['pressure']
    else:
        raise Exception(f"Weather API Request failed with status code: {response.status_code}")
    
    return (
        f"The current temperature in {city} is {temperature}°C with {weather_description}.\n"
        f"Humidity: {humidity}%, Wind Speed: {wind_speed} m/s, Pressure: {pressure} hPa."
    )



In [29]:
get_current_weather("Richmond")

'The current temperature in Richmond is 23.22°C with broken clouds.\nHumidity: 73%, Wind Speed: 4.12 m/s, Pressure: 1015 hPa.'

In [30]:
#Integrating the weather in the full code

import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.runnables import Runnable
import requests
from pydantic import BaseModel, Field

# Weather tool function
def get_current_weather(city: str) -> str:
    """Fetch current weather for a given city."""
    
    BASE_URL = "http://api.openweathermap.org/data/2.5/weather"
    
    params = {
        'q': city,
        'appid': OPENWEATHER_API_KEY,
        'units': 'metric',  # To get the temperature in Celsius
    }

    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
        temperature = results['main']['temp']
        weather_description = results['weather'][0]['description']
        humidity = results['main']['humidity']
        wind_speed = results['wind']['speed']
        pressure = results['main']['pressure']
    else:
        raise Exception(f"Weather API Request failed with status code: {response.status_code}")
    
    return (
        f"The current temperature in {city} is {temperature}°C with {weather_description}.\n"
        f"Humidity: {humidity}%, Wind Speed: {wind_speed} m/s, Pressure: {pressure} hPa."
    )

# Define the prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Define memory to store the previous exchanges
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Custom Runnable LLM class with debugging
class OllamaLLM(Runnable):
    def __init__(self, llm_fn):
        self.llm_fn = llm_fn

    def invoke(self, input, config=None, **kwargs):
        question = str(input)
        context = kwargs.get("context", "")
        print(f"Question passed to LLM: {question}")
        print(f"Context passed to LLM: {context}")
        stop = kwargs.get("stop", None)
        response = self.llm_fn(question, context)
        print(f"Response from LLM: {response}")
        return response

    def predict(self, input, **kwargs):
        return self.invoke(input, **kwargs)

    def __call__(self, *args, **kwargs):
        return self.invoke(*args, **kwargs)

# Instantiate the custom LLM class
ollama_llm_instance = OllamaLLM(ollama_llm)

# Define the conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ollama_llm_instance,
    retriever=retriever,
    memory=memory  # add the memory module to pass the previous exchange to the LLM as well
)

# Function to detect if the question is about the weather in a city
def is_weather_question(question: str) -> bool:
    return "weather" in question.lower() and "in" in question.lower()

# Extract city name from the weather question
def extract_city_from_question(question: str) -> str:
    # Simple heuristic to extract city name
    if "weather in" in question.lower():
        return question.lower().split("weather in")[1].strip().split()[0].capitalize()
    return ""

# Define the function to get important facts with debugging
def get_important_facts(question):
    # Check if the question is about the weather in a city
    if is_weather_question(question):
        city = extract_city_from_question(question)
        if city:
            return get_current_weather(city)
        else:
            return "I couldn't determine the city you're asking about. Please specify the city."
    
    # Otherwise, use the LLM-based chain
    print(f"Question passed to retriever: {question}")
    response = qa_chain.run({"question": question})
    print(f"Memory state: {memory.buffer}")
    return response

# Create a Gradio app interface
iface = gr.Interface(
    fn=get_important_facts,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
    outputs="text",
    title="RAG with Llama3",
    description="Ask questions",
)

# Launch the Gradio app
iface.launch()  # e.g. q: In which city did the Martians arrive?

INFO:httpx:HTTP Request: GET http://127.0.0.1:7869/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7869/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


## Working on the interface

In this example, we use gradio, you will find many other interface options, and in each of them possibilities to customize the look and feel. For example in Gradio, adding Chatbot look, with option to clear the chat if needed.


In [14]:
import gradio as gr
import ollama
from bs4 import BeautifulSoup as bs
from langchain_community.embeddings import OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.runnables import Runnable

# Define the prompt template
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum. Keep the answer as concise as possible. 

{context}

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

# Define memory to store the previous exchanges
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Custom Runnable LLM class with debugging
class OllamaLLM(Runnable):
    def __init__(self, llm_fn):
        self.llm_fn = llm_fn

    def invoke(self, input, config=None, **kwargs):
        question = str(input)  # Convert the input to a string if necessary
        context = kwargs.get("context", "")  # Retrieve context from kwargs if available

        # Debugging outputs
        print(f"Question passed to LLM: {question}")
        print(f"Context passed to LLM: {context}")

        stop = kwargs.get("stop", None)
        response = self.llm_fn(question, context)
        
        # Debugging output
        print(f"Response from LLM: {response}")

        return response

    def predict(self, input, **kwargs):
        return self.invoke(input, **kwargs)

    def __call__(self, *args, **kwargs):
        return self.invoke(*args, **kwargs)

# Instantiate the custom LLM class
ollama_llm_instance = OllamaLLM(ollama_llm)

# Define the conversational retrieval chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=ollama_llm_instance,
    retriever=retriever,
    memory=memory
)

# Define the function to get important facts with debugging
def get_important_facts(question, chat_history):
    print(f"Question passed to retriever: {question}")
    
    response = qa_chain.run({"question": question})
    print(f"Memory state: {memory.buffer}")
    
    # Append to chat history
    chat_history.append((question, response))
    return chat_history, chat_history

# Create a Gradio chat interface
with gr.Blocks() as iface:
    gr.Markdown("# RAG with Llama3")
    chatbot = gr.Chatbot(height=250)  # Adjust the height here
    question_input = gr.Textbox(lines=2, placeholder="Enter your question here...", show_label=False)
    submit_btn = gr.Button("Submit")
    clear_btn = gr.Button("Clear Chat")
    
    # Set up the interaction
    submit_btn.click(get_important_facts, [question_input, chatbot], [chatbot, chatbot])
    question_input.submit(get_important_facts, [question_input, chatbot], [chatbot, chatbot])
    clear_btn.click(lambda: None, None, chatbot, queue=False)

# Launch the Gradio app
iface.launch()
# e.g. q: Tell me about the Martian invasion

INFO:httpx:HTTP Request: GET http://127.0.0.1:7873/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7873/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7873

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


Question passed to retriever: Where did the Martians land?
Question passed to LLM: text="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nI was a lonely man, and they were very kind to me. I was a lonely man and a sad one, \nand they bore with me. I remained with them fo ur days after my rec overy. All that time I \nfelt a vague, a growing craving to look once more on whatever remained of the little life \nthat seemed so happy and bright in my past . It was a mere hopeless  desire to feast upon \nmy misery. They dissuaded me. They did all th ey could to divert me from this morbidity. \nBut at last I could resist the impulse no l onger, and, promising faithfully to return to \nthem, and parting, as I will confess, from thes e four-day friends with tears, I went out \nagain into the streets that had lately been so dark and strange and empty.\n\nand the contorted bodies shr

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


...

Response from LLM: The given text is from a novel by H.G. Wells, titled "The War of the Worlds". The context includes descriptions of the narrator's emotional state after recovering from an illness and his subsequent departure from four-day friends to revisit his past life. The narrative then shifts to a discussion about the Martian invasion, which has brought enormous scientific benefits but also led to humanity's loss of serene confidence in the future.

Please note that there is no description of the Martian rockets in this text. If you are looking for an answer based on this context, it would be "I don't know" because the text does not provide any information about the appearance or characteristics of the Martian rockets.
Memory state: [HumanMessage(content='Where did the Martians land?', additional_kwargs={}, response_metadata={}), AIMessage(content='The given text is a passage from H.G. Wells\' novel "The War of the Worlds". The story follows the narrator\'s account of the Martia

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response from LLM: The standalone follow-up question in its original language is:

Tell me about the Martian invasion.
Question passed to LLM: text='Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nAnd strangest of all is it to hold my wife \'s hand again, and to think that I have counted \nher, and that she has counted me, among the dead.\n\nthis planet as being fenced in and a secure abiding place for Man; we can never anticipate \nthe unseen good or evil that may come upon us suddenly out of space. It may be that in \nthe larger design of the universe this invasion from Mars is not  without its ultimate \nbenefit for men; it has robbed us of that serene conf idence in the future which is the most \nfruitful source of decadence, the gifts to human science it has brought are enormous, and \nit has done much to promote th e conception of the commonweal of mankind. It 

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Response from LLM: The context is a passage from H.G. Wells' science fiction novel "The War of the Worlds", specifically chapters 1 and 2. The passage describes the narrator's reactions to the Martian invasion, including his feelings about holding his wife's hand again after initially thinking she was dead, and his observations about the Martians' activities on Earth.
Memory state: [HumanMessage(content='Where did the Martians land?', additional_kwargs={}, response_metadata={}), AIMessage(content='The given text is a passage from H.G. Wells\' novel "The War of the Worlds". The story follows the narrator\'s account of the Martian invasion of Earth.\n\nSince there isn\'t any mention in this passage about where the Martians landed, I\'d say:\n\nHelpful Answer: I don\'t know', additional_kwargs={}, response_metadata={}), HumanMessage(content='What food did they eat?', additional_kwargs={}, response_metadata={}), AIMessage(content='Since we are given a passage from H.G. Wells\' novel "The W